# Crowd Head Detection + Counting (YOLOv8n)

Goal: fine-tune a YOLOv8 nano detector on the **RPEE-Heads** dataset, then use detections to estimate **crowd size (heads per frame)**.

This notebook is designed to run on **Kaggle GPU**.


In [ ]:
!pip -q install ultralytics


In [ ]:
import torch
from ultralytics import __version__ as ultralytics_version

print('ultralytics:', ultralytics_version)
print('torch:', torch.__version__)
print('cuda available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('gpu:', torch.cuda.get_device_name(0))


In [ ]:
import os

DATA_ROOT = "/kaggle/input/2024rpee-heads-dataset"
print(DATA_ROOT)
print(os.listdir(DATA_ROOT))


In [ ]:
import os, glob
from pathlib import Path

ROOT = "/kaggle/input/2024rpee-heads-dataset"

def find_images_dir(split_folder: str) -> str:
    # find any ".../<split_folder>/**/images" that also has a sibling "labels"
    candidates = glob.glob(f"{ROOT}/{split_folder}/**/images", recursive=True)
    for img_dir in candidates:
        lab_dir = img_dir.replace("/images", "/labels")
        if os.path.isdir(img_dir) and os.path.isdir(lab_dir):
            # return relative path from ROOT
            return os.path.relpath(img_dir, ROOT)
    raise FileNotFoundError(f"Could not find an images/labels pair under: {ROOT}/{split_folder}")

train_rel = find_images_dir("training")
val_rel   = find_images_dir("validation")
test_rel  = find_images_dir("testing")

print("Detected:")
print(" train:", train_rel)
print(" val  :", val_rel)
print(" test :", test_rel)

yaml_path = Path("/kaggle/working/rpee_heads.yaml")
yaml_path.write_text(f"""\
path: {ROOT}

train: {train_rel}
val: {val_rel}
test: {test_rel}

names: [head]
nc: 1
""")
print("\nWrote YAML:\n", yaml_path.read_text())


In [ ]:
!yolo detect train model=yolov8n.pt data=/kaggle/working/rpee_heads.yaml imgsz=832 epochs=80 batch=16 device=0


## Evaluate on the test split

In [ ]:
!yolo detect val model=/kaggle/working/runs/detect/train/weights/best.pt data=/kaggle/working/rpee_heads.yaml split=test imgsz=832


## Counting evaluation (MAE/RMSE of head counts)

## Counting metrics (what we measure)

For each image/frame:
- **GT count** = number of labeled head boxes in the YOLO label file
- **Pred count** = number of predicted boxes after confidence filtering + NMS
- **Error** = `pred - gt`

We report:
- **MAE**: average absolute error ("on average we miss/overcount by X heads")
- **RMSE**: penalizes large errors more strongly
- **Bias**: average signed error (negative = undercount, positive = overcount)

We also compute **bucketed errors** by crowd density (0–10, 11–30, …).


In [ ]:
from pathlib import Path

Path("/kaggle/working/eval_counting.py").write_text(r"""
import os
import glob
import math
from pathlib import Path
import pandas as pd
import numpy as np

from ultralytics import YOLO

def load_yolo_labels(txt_path: Path) -> int:
    if not txt_path.exists():
        return 0
    lines = [ln.strip() for ln in txt_path.read_text().splitlines() if ln.strip()]
    return len(lines)

def get_split_dirs(data_root: str, split: str):
    # data_root is YAML path root, split folder is e.g. 'testing' etc and maybe nested
    # We'll locate the first images dir that has sibling labels.
    candidates = glob.glob(os.path.join(data_root, split, "**", "images"), recursive=True)
    for img_dir in candidates:
        lab_dir = img_dir.replace("/images", "/labels")
        if os.path.isdir(img_dir) and os.path.isdir(lab_dir):
            return Path(img_dir), Path(lab_dir)
    raise FileNotFoundError(f"Could not find images/labels under {data_root}/{split}")

def eval_counts(model_path: str, data_root: str, split_folder: str, imgsz: int = 832, conf: float = 0.25, iou: float = 0.7, max_det: int = 300):
    img_dir, lab_dir = get_split_dirs(data_root, split_folder)

    model = YOLO(model_path)

    image_paths = sorted([p for p in img_dir.glob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png"]])
    rows = []

    # batch inference for speed
    results = model.predict(
        source=[str(p) for p in image_paths],
        imgsz=imgsz,
        conf=conf,
        iou=iou,
        max_det=max_det,
        device=0,
        verbose=False,
        stream=False,
    )

    for p, r in zip(image_paths, results):
        gt = load_yolo_labels(lab_dir / (p.stem + ".txt"))
        pred = 0 if r.boxes is None else int(r.boxes.shape[0])
        rows.append({"image": p.name, "gt_count": gt, "pred_count": pred, "error": pred - gt})

    df = pd.DataFrame(rows)
    mae = float(np.mean(np.abs(df["error"])))
    rmse = float(math.sqrt(np.mean(df["error"] ** 2)))
    bias = float(np.mean(df["error"]))

    # bucketed errors by crowd level
    bins = [-1, 10, 30, 60, 100, 200, 10**9]
    labels = ["0-10", "11-30", "31-60", "61-100", "101-200", "200+"]
    df["gt_bucket"] = pd.cut(df["gt_count"], bins=bins, labels=labels)
    bucket = df.groupby("gt_bucket", observed=True).agg(
        n=("image", "count"),
        mae=("error", lambda x: float(np.mean(np.abs(x)))),
        rmse=("error", lambda x: float(math.sqrt(np.mean(x**2)))),
        bias=("error", lambda x: float(np.mean(x))),
    ).reset_index()

    summary = {
        "split": split_folder,
        "n_images": int(len(df)),
        "mae": mae,
        "rmse": rmse,
        "bias": bias,
        "conf": conf,
        "iou": iou,
        "imgsz": imgsz,
        "max_det": max_det,
        "img_dir": str(img_dir),
    }
    return df, bucket, summary

def main():
    MODEL = "/kaggle/working/runs/detect/train/weights/best.pt"
    DATA_ROOT = "/kaggle/input/2024rpee-heads-dataset"
    OUT_DIR = Path("/kaggle/working/count_eval")
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    # Tune conf later; start with default 0.25
    for split in ["validation", "testing"]:
        df, bucket, summary = eval_counts(MODEL, DATA_ROOT, split_folder=split, imgsz=832, conf=0.25, iou=0.7, max_det=300)

        pd.DataFrame([summary]).to_csv(OUT_DIR / f"summary_{split}.csv", index=False)
        df.to_csv(OUT_DIR / f"per_image_{split}.csv", index=False)
        bucket.to_csv(OUT_DIR / f"bucket_{split}.csv", index=False)

        print(summary)
        print(bucket)

if __name__ == "__main__":
    main()
""")

!python /kaggle/working/eval_counting.py


## Scatter: predicted vs GT count + error histogram

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("/kaggle/working/count_eval/per_image_testing.csv")

# Scatter
plt.figure()
plt.scatter(df["gt_count"], df["pred_count"], s=6)
plt.xlabel("Ground truth head count")
plt.ylabel("Predicted head count")
plt.title("Head counting: Predicted vs Ground Truth (test)")
plt.savefig("/kaggle/working/count_eval/scatter_test.png", dpi=200, bbox_inches="tight")
plt.close()

# Error histogram
plt.figure()
plt.hist(df["error"], bins=60)
plt.xlabel("Prediction error (pred - gt)")
plt.ylabel("Number of images")
plt.title("Head counting error distribution (test)")
plt.savefig("/kaggle/working/count_eval/error_hist_test.png", dpi=200, bbox_inches="tight")
plt.close()

print("Saved plots to /kaggle/working/count_eval/")


## Tune conf for best counting MAE

In [ ]:
import pandas as pd
import numpy as np
from ultralytics import YOLO
import os, glob, math
from pathlib import Path

MODEL = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")
DATA_ROOT = "/kaggle/input/2024rpee-heads-dataset"

# locate validation images/labels
val_imgs = glob.glob(f"{DATA_ROOT}/validation/**/images/*.*", recursive=True)
val_imgs = [p for p in val_imgs if p.lower().endswith((".jpg",".jpeg",".png"))]
val_imgs = sorted(val_imgs)

val_labels_dir = None
# find sibling labels dir of first image
for img in val_imgs[:1]:
    d = os.path.dirname(img)
    cand = d.replace("/images", "/labels")
    if os.path.isdir(cand):
        val_labels_dir = cand
        break
assert val_labels_dir is not None

def gt_count(img_path):
    txt = os.path.join(val_labels_dir, Path(img_path).stem + ".txt")
    if not os.path.exists(txt):
        return 0
    with open(txt, "r") as f:
        return sum(1 for _ in f if _.strip())

gt = np.array([gt_count(p) for p in val_imgs], dtype=int)

def eval_conf(conf):
    res = MODEL.predict(val_imgs, imgsz=832, conf=conf, iou=0.7, device=0, verbose=False)
    pred = np.array([0 if r.boxes is None else int(r.boxes.shape[0]) for r in res], dtype=int)
    err = pred - gt
    mae = float(np.mean(np.abs(err)))
    bias = float(np.mean(err))
    rmse = float(math.sqrt(np.mean(err**2)))
    return mae, rmse, bias

grid = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35]
rows=[]
for c in grid:
    mae, rmse, bias = eval_conf(c)
    rows.append({"conf": c, "mae": mae, "rmse": rmse, "bias": bias})
df = pd.DataFrame(rows).sort_values("mae")
df.to_csv("/kaggle/working/count_eval/conf_sweep_val.csv", index=False)
df


## NMS IoU sweep

In [ ]:
import pandas as pd
import numpy as np
from ultralytics import YOLO
import os, glob, math
from pathlib import Path

MODEL = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")
DATA_ROOT = "/kaggle/input/2024rpee-heads-dataset"

val_imgs = sorted([p for p in glob.glob(f"{DATA_ROOT}/validation/**/images/*.*", recursive=True)
                   if p.lower().endswith((".jpg",".jpeg",".png"))])

val_labels_dir = None
for img in val_imgs[:1]:
    cand = os.path.dirname(img).replace("/images", "/labels")
    if os.path.isdir(cand):
        val_labels_dir = cand
        break
assert val_labels_dir is not None

def gt_count(img_path):
    txt = os.path.join(val_labels_dir, Path(img_path).stem + ".txt")
    if not os.path.exists(txt): return 0
    with open(txt, "r") as f:
        return sum(1 for ln in f if ln.strip())

gt = np.array([gt_count(p) for p in val_imgs], dtype=int)

def eval_iou(iou, conf=0.25):
    res = MODEL.predict(val_imgs, imgsz=832, conf=conf, iou=iou, device=0, verbose=False)
    pred = np.array([0 if r.boxes is None else int(r.boxes.shape[0]) for r in res], dtype=int)
    err = pred - gt
    mae = float(np.mean(np.abs(err)))
    bias = float(np.mean(err))
    rmse = float(math.sqrt(np.mean(err**2)))
    return mae, rmse, bias

grid = [0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75]
rows = []
for i in grid:
    mae, rmse, bias = eval_iou(i)
    rows.append({"iou": i, "conf": 0.25, "mae": mae, "rmse": rmse, "bias": bias})

df = pd.DataFrame(rows).sort_values("mae")
df.to_csv("/kaggle/working/count_eval/iou_sweep_val.csv", index=False)
df


## Compare bucket errors at iou=0.70 vs 0.75

 real pain is the 101–200 bucket. The global MAE can hide whether 0.75 helps that bucket.

Run this for both val and test and compare only the dense bucket:

In [ ]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
import os, glob, math
from pathlib import Path

MODEL_PATH = "/kaggle/working/runs/detect/train/weights/best.pt"
DATA_ROOT  = "/kaggle/input/2024rpee-heads-dataset"

def find_pair(split_folder):
    imgs = glob.glob(f"{DATA_ROOT}/{split_folder}/**/images/*.*", recursive=True)
    imgs = sorted([p for p in imgs if p.lower().endswith((".jpg",".jpeg",".png"))])
    for p in imgs[:1]:
        lab = os.path.dirname(p).replace("/images", "/labels")
        if os.path.isdir(lab):
            return imgs, lab
    raise FileNotFoundError(split_folder)

def gt_count(labels_dir, img_path):
    txt = os.path.join(labels_dir, Path(img_path).stem + ".txt")
    if not os.path.exists(txt): return 0
    with open(txt, "r") as f:
        return sum(1 for ln in f if ln.strip())

def bucket_eval(split_folder, conf, iou):
    imgs, lab_dir = find_pair(split_folder)
    model = YOLO(MODEL_PATH)
    gt = np.array([gt_count(lab_dir, p) for p in imgs], dtype=int)

    res = model.predict(imgs, imgsz=832, conf=conf, iou=iou, max_det=300, device=0, verbose=False)
    pred = np.array([0 if r.boxes is None else int(r.boxes.shape[0]) for r in res], dtype=int)

    err = pred - gt
    df = pd.DataFrame({"gt": gt, "pred": pred, "err": err})

    bins = [-1, 10, 30, 60, 100, 200, 10**9]
    labels = ["0-10", "11-30", "31-60", "61-100", "101-200", "200+"]
    df["bucket"] = pd.cut(df["gt"], bins=bins, labels=labels)

    out = df.groupby("bucket", observed=True).agg(
        n=("gt", "count"),
        mae=("err", lambda x: float(np.mean(np.abs(x)))),
        rmse=("err", lambda x: float(math.sqrt(np.mean(x**2)))),
        bias=("err", lambda x: float(np.mean(x))),
    ).reset_index()

    overall = {
        "split": split_folder, "conf": conf, "iou": iou,
        "mae": float(np.mean(np.abs(err))),
        "rmse": float(math.sqrt(np.mean(err**2))),
        "bias": float(np.mean(err)),
    }
    return pd.DataFrame([overall]), out

for iou in [0.70, 0.75]:
    for split in ["validation", "testing"]:
        overall, bucket = bucket_eval(split, conf=0.25, iou=iou)
        print("\n", overall.to_string(index=False))
        print(bucket.to_string(index=False))


## Calibration of counts

## Calibration experiment (and why we keep it as analysis, not as production)

We tried **isotonic regression** to map `pred_count -> corrected_count` using the validation set.

- It **improved validation MAE** (because it can overfit the validation distribution).
- It **hurt test MAE** and shifted error between buckets (classic "calibration overfit" / distribution shift).

Conclusion for a robust baseline:
- Prefer tuning **confidence threshold** and **NMS IoU**
- If we calibrate, we do it with stronger regularization and/or more representative data from the target cameras


In [ ]:
import pandas as pd
import numpy as np
from sklearn.isotonic import IsotonicRegression

val  = pd.read_csv("/kaggle/working/count_eval/per_image_validation.csv")
test = pd.read_csv("/kaggle/working/count_eval/per_image_testing.csv")

# Fit on validation only
iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(val["pred_count"].values, val["gt_count"].values)

# Apply (rounded to integer counts, no negatives)
val["pred_count_cal"]  = np.maximum(0, np.rint(iso.predict(val["pred_count"].values)).astype(int))
test["pred_count_cal"] = np.maximum(0, np.rint(iso.predict(test["pred_count"].values)).astype(int))

val.to_csv("/kaggle/working/count_eval/per_image_validation_calibrated.csv", index=False)
test.to_csv("/kaggle/working/count_eval/per_image_testing_calibrated.csv", index=False)

print("Saved calibrated CSVs.")
print("val columns:", list(val.columns))


In [ ]:
import numpy as np
import pandas as pd
import math

def metrics(df, pred_col):
    err = df[pred_col].values - df["gt_count"].values
    return {
        "mae": float(np.mean(np.abs(err))),
        "rmse": float(math.sqrt(np.mean(err**2))),
        "bias": float(np.mean(err))
    }

val  = pd.read_csv("/kaggle/working/count_eval/per_image_validation_calibrated.csv")
test = pd.read_csv("/kaggle/working/count_eval/per_image_testing_calibrated.csv")

rows = []
rows.append({"split":"validation","version":"raw", **metrics(val, "pred_count")})
rows.append({"split":"validation","version":"cal", **metrics(val, "pred_count_cal")})
rows.append({"split":"testing","version":"raw", **metrics(test, "pred_count")})
rows.append({"split":"testing","version":"cal", **metrics(test, "pred_count_cal")})

pd.DataFrame(rows)


In [ ]:
import pandas as pd
import numpy as np
import math

def bucket_table(df, pred_col):
    gt = df["gt_count"].values
    pred = df[pred_col].values
    err = pred - gt

    bins = [-1, 10, 30, 60, 100, 200, 10**9]
    labels = ["0-10", "11-30", "31-60", "61-100", "101-200", "200+"]
    b = pd.cut(gt, bins=bins, labels=labels)

    out = pd.DataFrame({"bucket": b, "err": err}).groupby("bucket", observed=True).agg(
        n=("err","count"),
        mae=("err", lambda x: float(np.mean(np.abs(x)))),
        rmse=("err", lambda x: float(math.sqrt(np.mean(x**2)))),
        bias=("err", lambda x: float(np.mean(x))),
    ).reset_index()
    return out

val  = pd.read_csv("/kaggle/working/count_eval/per_image_validation_calibrated.csv")
test = pd.read_csv("/kaggle/working/count_eval/per_image_testing_calibrated.csv")

print("TEST raw buckets")
print(bucket_table(test, "pred_count").to_string(index=False))
print("\nTEST calibrated buckets")
print(bucket_table(test, "pred_count_cal").to_string(index=False))


In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.isotonic import IsotonicRegression

MAX_DET = 300  # must match the inference max_det

val  = pd.read_csv("/kaggle/working/count_eval/per_image_validation.csv")
test = pd.read_csv("/kaggle/working/count_eval/per_image_testing.csv")

x = val["pred_count"].values.astype(float)
y = val["gt_count"].values.astype(float)

# Anchors to make extrapolation sane
x_aug = np.concatenate([x, [0.0, float(MAX_DET)]])
y_aug = np.concatenate([y, [0.0, float(MAX_DET)]])

iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(x_aug, y_aug)

def apply_iso(df):
    cal = iso.predict(df["pred_count"].values.astype(float))
    df = df.copy()
    df["pred_count_cal"] = np.clip(np.rint(cal), 0, MAX_DET).astype(int)
    return df

val_cal  = apply_iso(val)
test_cal = apply_iso(test)

val_cal.to_csv("/kaggle/working/count_eval/per_image_validation_calibrated_anchored.csv", index=False)
test_cal.to_csv("/kaggle/working/count_eval/per_image_testing_calibrated_anchored.csv", index=False)

def metrics(df, pred_col):
    err = df[pred_col].values - df["gt_count"].values
    return {
        "mae": float(np.mean(np.abs(err))),
        "rmse": float(math.sqrt(np.mean(err**2))),
        "bias": float(np.mean(err)),
    }

summary = pd.DataFrame([
    {"split":"validation","version":"raw", **metrics(val, "pred_count")},
    {"split":"validation","version":"cal_anchored", **metrics(val_cal, "pred_count_cal")},
    {"split":"testing","version":"raw", **metrics(test, "pred_count")},
    {"split":"testing","version":"cal_anchored", **metrics(test_cal, "pred_count_cal")},
])
summary


In [ ]:
def bucket_table(df, pred_col):
    gt = df["gt_count"].values
    pred = df[pred_col].values
    err = pred - gt

    bins = [-1, 10, 30, 60, 100, 200, 10**9]
    labels = ["0-10", "11-30", "31-60", "61-100", "101-200", "200+"]
    b = pd.cut(gt, bins=bins, labels=labels)

    out = pd.DataFrame({"bucket": b, "err": err}).groupby("bucket", observed=True).agg(
        n=("err","count"),
        mae=("err", lambda x: float(np.mean(np.abs(x)))),
        rmse=("err", lambda x: float(math.sqrt(np.mean(x**2)))),
        bias=("err", lambda x: float(np.mean(x))),
    ).reset_index()
    return out

print("TEST raw buckets")
print(bucket_table(test, "pred_count").to_string(index=False))
print("\nTEST anchored-cal buckets")
print(bucket_table(test_cal, "pred_count_cal").to_string(index=False))


In [ ]:
import shutil

src = "/kaggle/working/runs"
dst = "/kaggle/working/runs"  # no .zip here
shutil.make_archive(dst, "zip", src)

print(dst + ".zip")

# Publish to Hugging Face

This section packages the trained model (`best.pt`) and creates:
- a **Model repo** (stores weights + minimal inference notes)
- an optional **Space** (Gradio demo: upload image -> head count + annotated image)

It will need a Hugging Face access token with write permissions.


In [1]:
!pip -q install huggingface_hub gradio opencv-python


In [3]:
import os
from huggingface_hub import HfApi, login

# Option A: set HF_TOKEN as a Kaggle "Secret" and read it from env
# Option B: paste token here (not recommended)
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

hf_token = os.environ.get('HF_TOKEN')
assert secret_value_0, 'Missing HF_TOKEN env var. Add it in Kaggle secrets or os.environ.'
login(token=secret_value_0)
print('Logged in to Hugging Face')


Logged in to Hugging Face


## 1) Create a model repo and upload the weights



In [5]:
from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()

HF_MODEL_REPO = 'AmineSam/irail-crowd-counting-yolov8n'
LOCAL_WEIGHTS = Path('/kaggle/working/runs/detect/train/weights/best.pt')
assert LOCAL_WEIGHTS.exists(), f'Missing weights: {LOCAL_WEIGHTS}'

api.create_repo(repo_id=HF_MODEL_REPO, repo_type='model', exist_ok=True)
api.upload_file(
    path_or_fileobj=str(LOCAL_WEIGHTS),
    path_in_repo='best.pt',
    repo_id=HF_MODEL_REPO,
    repo_type='model',
)

# Optional: upload key plots/CSVs for transparency
for p in [
    Path('/kaggle/working/count_eval/scatter_test.png'),
    Path('/kaggle/working/count_eval/error_hist_test.png'),
]:
    if p.exists():
        api.upload_file(
            path_or_fileobj=str(p),
            path_in_repo=f"artifacts/{p.name}",
            repo_id=HF_MODEL_REPO,
            repo_type="model",
        )


print('Uploaded model + artifacts to:', HF_MODEL_REPO)


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Uploaded model + artifacts to: AmineSam/irail-crowd-counting-yolov8n


## 2) Create a Gradio Space (optional)

This Space will:
- load `best.pt` from the model repo
- run YOLO inference
- display an annotated image and the head count



In [6]:
from pathlib import Path

SPACE_DIR = Path('/kaggle/working/hf_space')
SPACE_DIR.mkdir(parents=True, exist_ok=True)

(SPACE_DIR / 'requirements.txt').write_text('''
import os
import cv2
import gradio as gr
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

MODEL_REPO = "AmineSam/irail-crowd-counting-yolov8n"

# Fixed inference params (not exposed in UI)
DEFAULT_IMGSZ = 832
DEFAULT_MAX_DET = 300

# Crowd thresholds (based on the dataset buckets)
LOW_MAX = 40
MED_MAX = 100

# Download weights from the model repo (public repo -> no token needed)
weights_path = hf_hub_download(repo_id=MODEL_REPO, filename="best.pt")
model = YOLO(weights_path)


def crowd_level_from_count(count: int) -> str:
    if count <= LOW_MAX:
        return f"Low (0–{LOW_MAX})"
    if count <= MED_MAX:
        return f"Medium ({LOW_MAX+1}–{MED_MAX})"
    return f"High (>{MED_MAX})"


def predict(image, conf=0.25, iou=0.75):
    if image is None:
        return None, 0, "N/A"

    # image is RGB numpy
    bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    res = model.predict(
        bgr,
        conf=float(conf),
        iou=float(iou),
        imgsz=DEFAULT_IMGSZ,
        max_det=DEFAULT_MAX_DET,
        verbose=False,
    )[0]

    count = 0 if res.boxes is None else int(res.boxes.shape[0])
    level = crowd_level_from_count(count)

    annotated = res.plot()  # BGR annotated
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

    return annotated, count, level


EXAMPLES = [
    ["DMPG41_GP204092_frame_1500.jpg"],
    ["DMSA0442_GP094171_frame_11250.jpg"],
    ["DMWW71_MitsubishiElectricHalle_WincentWeiss_gopro7_1_1430to1915_part17_13680032_14535034_frame_4500.jpg"],
    ["DMWW71_MitsubishiElectricHalle_WincentWeiss_gopro7_1_1430to1915_part17_13680032_14535034_frame_12000.jpg"],
]

with gr.Blocks() as demo:
    gr.Markdown("# RPEE-Heads: Head detection + counting")
    gr.Markdown(
        f"**Crowd levels** (based on dataset buckets): Low ≤ {LOW_MAX}, Medium {LOW_MAX+1}–{MED_MAX}, High ≥ {MED_MAX+1}"
    )

    with gr.Row():
        inp = gr.Image(type="numpy", label="Input image")
        out = gr.Image(type="numpy", label="Annotated output")

    with gr.Row():
        count = gr.Number(label="Predicted head count")
        level = gr.Textbox(label="Crowd level", interactive=False)

    with gr.Accordion("Inference settings", open=False):
        conf = gr.Slider(0.05, 0.80, value=0.25, step=0.05, label="Confidence")
        iou = gr.Slider(0.30, 0.90, value=0.75, step=0.05, label="NMS IoU")

    btn = gr.Button("Run")
    btn.click(fn=predict, inputs=[inp, conf, iou], outputs=[out, count, level])

    gr.Examples(
        examples=EXAMPLES,
        inputs=[inp],
        label="Example images",
    )

demo.launch()
''')

print('Wrote Space files to:', SPACE_DIR)


Wrote Space files to: /kaggle/working/hf_space


In [7]:
from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
HF_SPACE_REPO = 'AmineSam/irail-crowd-counting-yolov8n-demo'

api.create_repo(repo_id=HF_SPACE_REPO, repo_type='space', space_sdk='gradio', exist_ok=True)
api.upload_folder(folder_path=str(Path('/kaggle/working/hf_space')), repo_id=HF_SPACE_REPO, repo_type='space')

print('Uploaded Space to:', HF_SPACE_REPO)
print('Tip: set Space env var HF_MODEL_REPO to the model repo id.')


Uploaded Space to: AmineSam/irail-crowd-counting-yolov8n-demo
Tip: set Space env var HF_MODEL_REPO to your model repo id.
